# Tutorial: Resource Leak Detection with LeakTracker

**Category**: Concurrency  
**Difficulty**: Intermediate  
**Time**: 25 minutes

## Overview

Resource leaks (unclosed files, unreleased connections, forgotten locks) cause production failures that are difficult to debug. lionherd-core's `LeakTracker` provides automatic leak detection for any resource type.

**What You'll Learn**:
- Track resources with `LeakTracker` API
- Detect leaks automatically at program exit
- Generate detailed leak reports
- Integrate with context managers for guaranteed cleanup

**Prerequisites**:
```bash
pip install lionherd-core
```

In [1]:
# Standard library
import atexit
import tempfile
import time
from collections import deque
from dataclasses import dataclass
from pathlib import Path

# lionherd-core
from lionherd_core.libs.concurrency import LeakTracker, sleep

## Section 1: LeakTracker Basics

**API Overview**:
- `LeakTracker()` - Create isolated tracker instance
- `tracker.track(obj, name, kind)` - Register resource for tracking
- `tracker.untrack(obj)` - Remove resource (cleanup completed)
- `tracker.live()` - Get list of currently tracked resources
- `tracker.clear()` - Remove all tracked resources

**Important**: Only objects supporting weak references can be tracked (custom classes work, built-in types like `dict`/`list` don't).

**When to Use**:
- ✅ Finite OS resources (file descriptors, connections, locks)
- ✅ Resources that must be explicitly released
- ✅ Production systems where leaks cause outages
- ❌ Memory-only objects (use memory profiler instead)

In [2]:
# Example 1: Basic tracking
tracker = LeakTracker()


# Create trackable resource (must support weak references)
class Resource:
    def __init__(self, resource_id: int, data: str):
        self.resource_id = resource_id
        self.data = data


resource = Resource(1, "example")
tracker.track(resource, name="resource-1", kind="custom")

print(f"Tracked resources: {len(tracker.live())}")
print(f"Resource info: {tracker.live()[0]}")

# Cleanup - untrack when done
tracker.untrack(resource)
print(f"After cleanup: {len(tracker.live())} resources")

Tracked resources: 1
Resource info: LeakInfo(name='resource-1', kind='custom', created_at=1762746199.831727)
After cleanup: 0 resources


**Key Points**:
- **Isolated tracker**: Each `LeakTracker()` instance is independent
- **Metadata**: `name` and `kind` help identify leaked resources in reports
- **Timestamp**: `created_at` automatically recorded for age calculation
- **Thread-safe**: Uses `threading.Lock` internally

## Section 2: File Handle Tracking

**Problem**: File descriptors are limited (256-4096 per process). Unclosed files cause "Too many open files" crashes.

**Solution**: Track file handles with `LeakTracker`, detect leaks at program exit.

In [3]:
# Example 2: File handle tracking
class TrackedFile:
    """File wrapper with automatic leak detection."""

    _tracker = LeakTracker()  # Shared tracker for all files

    def __init__(self, path: str, mode: str = "r"):
        self.path = path
        self.mode = mode
        self.file = open(path, mode)

        # Track this file handle
        self._tracker.track(self, name=f"file:{path}", kind=f"file_{mode}")

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()
        return False

    def close(self):
        if hasattr(self, "file") and self.file:
            self.file.close()
            self._tracker.untrack(self)  # Remove from tracking
            self.file = None

    def read(self) -> str:
        return self.file.read()

    def write(self, data: str) -> int:
        return self.file.write(data)

    @classmethod
    def check_leaks(cls):
        """Get currently open files."""
        return cls._tracker.live()


# Create temp file for testing
with tempfile.NamedTemporaryFile(mode="w", delete=False, suffix=".txt") as tf:
    test_file = tf.name
    tf.write("test data\n")

# Scenario 1: Proper usage with context manager
with TrackedFile(test_file, "r") as f:
    data = f.read()
    print(f"Read: {data.strip()}")

print(f"After context manager: {len(TrackedFile.check_leaks())} open files")

# Scenario 2: Leak (forgot to close)
leaked = TrackedFile(test_file, "r")
data = leaked.read()
# Oops! Forgot to call leaked.close()

print(f"After leak: {len(TrackedFile.check_leaks())} open files")
print(f"Leaked file: {TrackedFile.check_leaks()[0].name}")

# Cleanup
leaked.close()
Path(test_file).unlink()

Read: test data
After context manager: 0 open files
After leak: 1 open files
Leaked file: file:/var/folders/5p/rcbw097d29j3s2qt861tsjfh0000gn/T/tmpwku6j9u8.txt


**Pattern**:
1. Track in `__init__` when resource acquired
2. Untrack in `close()` when resource released
3. Use context manager (`with`) for automatic cleanup
4. Check `tracker.live()` to detect leaks

## Section 3: Connection Pool Tracking

**Problem**: Connection pools exhaust when connections aren't returned. Hard to debug which code path leaked.

**Solution**: Track acquired connections, detect which aren't released.

In [4]:
# Example 3: Connection pool tracking
@dataclass
class Connection:
    """Mock database connection."""

    conn_id: int
    host: str
    port: int

    async def execute(self, query: str) -> list:
        await sleep(0.01)  # Simulate query
        return [{"result": f"Query: {query}"}]


class TrackedConnectionPool:
    """Connection pool with leak detection."""

    def __init__(self, host: str, port: int, pool_size: int = 3):
        self.host = host
        self.port = port
        self._available = deque([Connection(i, host, port) for i in range(pool_size)])
        self._in_use = set()
        self._tracker = LeakTracker()  # Track acquired connections

    async def acquire(self) -> Connection:
        """Get connection from pool."""
        if not self._available:
            raise RuntimeError("Pool exhausted")

        conn = self._available.popleft()
        self._in_use.add(conn.conn_id)

        # Track acquisition
        self._tracker.track(conn, name=f"conn-{conn.conn_id}", kind="database_connection")

        return conn

    async def release(self, conn: Connection):
        """Return connection to pool."""
        if conn.conn_id not in self._in_use:
            raise ValueError(f"Connection {conn.conn_id} not from this pool")

        # Untrack before returning to pool
        self._tracker.untrack(conn)
        self._in_use.remove(conn.conn_id)
        self._available.append(conn)

    def leak_report(self) -> str:
        """Generate leak report."""
        leaks = self._tracker.live()

        if not leaks:
            return "✅ No connection leaks"

        lines = [f"⚠️  {len(leaks)} connection(s) leaked:", ""]
        now = time.time()
        for info in sorted(leaks, key=lambda x: x.created_at):
            age = now - info.created_at
            lines.append(f"  • {info.name} (held for {age:.2f}s)")

        return "\n".join(lines)


# Test the pool
pool = TrackedConnectionPool("localhost", 5432, pool_size=3)

# Scenario 1: Proper usage
conn1 = await pool.acquire()
result = await conn1.execute("SELECT 1")
await pool.release(conn1)
print("After proper release:")
print(pool.leak_report())
print()

# Scenario 2: Leak (acquire without release)
conn2 = await pool.acquire()
conn3 = await pool.acquire()
await conn2.execute("SELECT 2")
# Oops! Forgot to release conn2 and conn3

print("After leaking 2 connections:")
print(pool.leak_report())

# Cleanup
await pool.release(conn2)
await pool.release(conn3)

After proper release:
✅ No connection leaks

After leaking 2 connections:
⚠️  2 connection(s) leaked:

  • conn-1 (held for 0.01s)
  • conn-2 (held for 0.01s)


**Key Points**:
- **Pool-specific tracker**: Each pool has isolated `LeakTracker` instance
- **Track on acquire**: Register when resource leaves pool
- **Untrack on release**: Remove when resource returns to pool
- **Age tracking**: `created_at` shows how long connection held

## Section 4: Exit-Time Leak Detection

**Problem**: Manual leak checking requires developer discipline. Need automatic detection.

**Solution**: Use Python's `atexit` to automatically report leaks at program termination.

In [5]:
# Example 4: Automatic leak reporting at exit
class ManagedFile:
    """File with automatic exit-time leak detection."""

    _tracker = LeakTracker()
    _atexit_registered = False

    def __init__(self, path: str, mode: str = "r"):
        self.path = path
        self.mode = mode
        self.file = open(path, mode)

        # Register exit handler once (first file open)
        # Pattern: Use class variable flag to ensure single registration
        if not ManagedFile._atexit_registered:
            atexit.register(ManagedFile._report_leaks_at_exit)
            ManagedFile._atexit_registered = True

        self._tracker.track(self, name=f"file:{path}", kind=f"file_{mode}")

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()
        return False

    def close(self):
        if hasattr(self, "file") and self.file:
            self.file.close()
            self._tracker.untrack(self)
            self.file = None

    def read(self) -> str:
        return self.file.read()

    @classmethod
    def _report_leaks_at_exit(cls):
        """Called automatically at program exit."""
        leaks = cls._tracker.live()

        if not leaks:
            return  # Silent success

        # Print detailed leak report
        print(f"\n{'=' * 60}")
        print(f"[LEAK DETECTION] ⚠️  {len(leaks)} file(s) not closed")
        print(f"{'=' * 60}")

        now = time.time()
        for info in sorted(leaks, key=lambda x: x.created_at):
            age = now - info.created_at
            print(f"  • {info.name}")
            print(f"    Mode: {info.kind}, Age: {age:.2f}s")

        print("\nFix: Use 'with ManagedFile(...) as f:' pattern")
        print(f"{'=' * 60}\n")


# Demonstrate exit-time detection
with tempfile.NamedTemporaryFile(mode="w", delete=False, suffix="_test.txt") as tf:
    test_path = tf.name
    tf.write("leak detection test\n")

# Proper usage
with ManagedFile(test_path, "r") as f:
    print(f"Read: {f.read().strip()}")

# Create leak
leaked = ManagedFile(test_path, "r")
leaked.read()
# Forgot to close!

# Simulate program exit
print("\n--- Simulating program exit ---")
ManagedFile._report_leaks_at_exit()

# Cleanup
leaked.close()
Path(test_path).unlink()

Read: leak detection test

--- Simulating program exit ---

[LEAK DETECTION] ⚠️  1 file(s) not closed
  • file:/var/folders/5p/rcbw097d29j3s2qt861tsjfh0000gn/T/tmpnc7iw2ox_test.txt
    Mode: file_r, Age: 0.00s

Fix: Use 'with ManagedFile(...) as f:' pattern



**Pattern**:
- **Register once**: `atexit.register()` called on first resource creation
- **Class variable flag**: `_atexit_registered` prevents duplicate registrations across instances
- **Exit-time execution**: Handler runs during interpreter shutdown
- **Automatic reporting**: Developers see leaks without manual checking
- **Silent success**: No output if all resources properly cleaned up

## Section 5: Enhanced Leak Reports

Production systems need rich leak reports: grouping by type, statistics, and prioritization.

In [6]:
# Example 5: Rich leak reporting
@dataclass
class LeakReport:
    """Structured leak report."""

    total_leaks: int
    by_kind: dict[str, int]
    oldest_age: float
    leaks: list


def generate_leak_report(tracker: LeakTracker) -> LeakReport:
    """Generate detailed leak report from tracker."""
    leaks = tracker.live()

    if not leaks:
        return LeakReport(total_leaks=0, by_kind={}, oldest_age=0.0, leaks=[])

    # Group by resource kind
    by_kind = {}
    for info in leaks:
        kind = info.kind or "unknown"
        by_kind[kind] = by_kind.get(kind, 0) + 1

    # Calculate oldest leak age
    now = time.time()
    oldest_age = max((now - info.created_at) for info in leaks)

    # Sort by age (oldest first - highest priority)
    sorted_leaks = sorted(leaks, key=lambda x: x.created_at)

    return LeakReport(
        total_leaks=len(leaks), by_kind=by_kind, oldest_age=oldest_age, leaks=sorted_leaks
    )


def print_leak_report(report: LeakReport):
    """Print formatted leak report."""
    if report.total_leaks == 0:
        print("✅ No leaks detected")
        return

    print("⚠️  Leak Summary:")
    print(f"  Total: {report.total_leaks}")
    print(f"  Oldest: {report.oldest_age:.2f}s ago")
    print("\n  By type:")
    for kind, count in sorted(report.by_kind.items()):
        print(f"    {kind}: {count}")

    print("\n  Details (oldest first):")
    now = time.time()
    for info in report.leaks:
        age = now - info.created_at
        print(f"    • {info.name} ({age:.2f}s)")


# Create mixed resource leaks for demonstration
tracker = LeakTracker()


# Define trackable resource types
class MockConnection:
    def __init__(self, conn_id: int):
        self.conn_id = conn_id


class MockFile:
    def __init__(self, file_id: int):
        self.file_id = file_id


class MockLock:
    def __init__(self, lock_id: int):
        self.lock_id = lock_id


# Create resources
resources = [
    (MockConnection(1), "conn-1", "database_connection"),
    (MockFile(1), "file-1", "file_handle"),
    (MockConnection(2), "conn-2", "database_connection"),
    (MockLock(1), "lock-1", "mutex_lock"),
    (MockFile(2), "file-2", "file_handle"),
]

for obj, name, kind in resources:
    tracker.track(obj, name=name, kind=kind)
    time.sleep(0.05)  # Different ages

# Generate and print report
report = generate_leak_report(tracker)
print_leak_report(report)

# Cleanup
tracker.clear()

⚠️  Leak Summary:
  Total: 5
  Oldest: 0.27s ago

  By type:
    database_connection: 2
    file_handle: 2
    mutex_lock: 1

  Details (oldest first):
    • conn-1 (0.27s)
    • file-1 (0.21s)
    • conn-2 (0.16s)
    • lock-1 (0.11s)
    • file-2 (0.06s)


**Report Features**:
- **Grouping by kind**: Identify patterns ("all database connections leak")
- **Age-based sorting**: Oldest leaks appear first (likely systematic issues)
- **Summary statistics**: Quick overview of leak severity
- **Structured data**: `LeakReport` dataclass for programmatic access

## Summary

### lionherd-core LeakTracker API

| Method | Purpose | When to Use |
|--------|---------|-------------|
| `LeakTracker()` | Create tracker | One per resource type or pool |
| `track(obj, name, kind)` | Register resource | When resource acquired |
| `untrack(obj)` | Remove resource | When resource released |
| `live()` | Get active resources | Leak detection, reporting |
| `clear()` | Remove all tracked | Shutdown, cleanup |

### Integration Patterns

1. **Context managers**: `__exit__` calls `untrack()` for guaranteed cleanup
2. **Exit handlers**: `atexit.register()` for automatic leak reporting
3. **Pool integration**: Track acquire/release in pool methods
4. **Rich reporting**: Group by kind, sort by age, generate statistics

### When to Use LeakTracker

- ✅ File descriptors (limited OS resources)
- ✅ Database connections (pool exhaustion causes outages)
- ✅ Network sockets (OS limits, port exhaustion)
- ✅ Locks and semaphores (deadlock debugging)
- ❌ Pure memory objects (use memory profiler instead)
- ❌ Unlimited resources (tracking overhead not justified)

### Key Takeaways

1. **LeakTracker uses weak references**: Automatic cleanup when objects deleted
2. **Track on acquire, untrack on release**: Simple integration pattern
3. **Isolated trackers**: One per resource type for clean separation
4. **Exit-time detection**: Last-line defense catches bugs before production
5. **Overhead is minimal**: Track/untrack operations are fast relative to I/O

### Related Resources

- [API Reference: LeakTracker](../../docs/api/libs/concurrency/resource_tracker.md)
- [Tutorial: Transaction Shielding](./)